<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view>

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd

## Data Wrangling

In [2]:
df = pd.read_csv('teledata.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Define features and targets
features = df.columns.values.tolist()
features.remove('customerID')
features.remove('Churn')

In [4]:
df[features].head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65
5,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5
6,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4
7,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9
8,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05
9,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95


In [5]:
from sklearn.preprocessing import OrdinalEncoder

In [6]:
encoder = OrdinalEncoder()

In [7]:
# Select variables to be encoded
to_be_encoded = ['gender', 'Partner', 'Dependents', 'PhoneService',
                 'MultipleLines', 'InternetService', 'OnlineSecurity',
                 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                 'StreamingTV', 'StreamingMovies', 'Contract',
                 'PaperlessBilling', 'PaymentMethod', 'Churn']

In [8]:
# Encode
data_encoded = pd.DataFrame(encoder.fit_transform(df[to_be_encoded]), columns=to_be_encoded)

In [9]:
# Add back on the numbers
data = data_encoded.join(df['SeniorCitizen']).join(df['tenure'])

In [10]:
data.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn,SeniorCitizen,tenure
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0,1
1,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,34
2,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0,2
3,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0,45
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0,2


In [11]:
data['MonthlyCharges'] = df['MonthlyCharges'].apply(float)

In [12]:
features.remove('TotalCharges')

In [13]:
X = data[features]
y = data['Churn']

In [14]:
from tensorflow.keras.utils import normalize

In [15]:
# Normalize
X = normalize(X)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
X.shape

(7043, 18)

## Baseline Model

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Model Building Function
def create_model():
    model = Sequential()
    model.add(Dense(4, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Complile Model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [26]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

In [27]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
4718/4718 [==============================] - 0s 62us/sample - loss: 0.6894 - acc: 0.5589
Epoch 2/20
4718/4718 [==============================] - 0s 45us/sample - loss: 0.6522 - acc: 0.7370
Epoch 3/20
4718/4718 [==============================] - 0s 40us/sample - loss: 0.6311 - acc: 0.7370
Epoch 4/20
4718/4718 [==============================] - 0s 51us/sample - loss: 0.6145 - acc: 0.7370
Epoch 5/20
4718/4718 [==============================] - 0s 49us/sample - loss: 0.6011 - acc: 0.7370
Epoch 6/20
4718/4718 [==============================] - 0s 49us/sample - loss: 0.5898 - acc: 0.7370
Epoch 7/20
4718/4718 [==============================] - 0s 66us/sample - loss: 0.5798 - acc: 0.7370
Epoch 8/20
4718/4718 [==============================] - 0s 100us/sample - loss: 0.5700 - acc: 0.7370
Epoch 9/20
4718/4718 [==============================] - 0s 83us/sample - loss: 0.5600 - acc: 0.7370
Epoch 10/20
4718/4718 [==============================] - 0s 48us/sample - loss: 0.5498 - acc: 0.737

## Tune batch_size

In [28]:
# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10]}

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X_train, y_train)

Epoch 1/10
3145/3145 [==============================] - 1s 164us/sample - loss: 0.6011 - acc: 0.7345
Epoch 2/10
3145/3145 [==============================] - 0s 150us/sample - loss: 0.5583 - acc: 0.7345
Epoch 3/10
3145/3145 [==============================] - 0s 137us/sample - loss: 0.5316 - acc: 0.7345
Epoch 4/10
3145/3145 [==============================] - 0s 138us/sample - loss: 0.5068 - acc: 0.7345
Epoch 5/10
3145/3145 [==============================] - 1s 277us/sample - loss: 0.4854 - acc: 0.7377
Epoch 6/10
3145/3145 [==============================] - 1s 198us/sample - loss: 0.4704 - acc: 0.7625
Epoch 7/10
3145/3145 [==============================] - 1s 203us/sample - loss: 0.4621 - acc: 0.7844
Epoch 8/10
3145/3145 [==============================] - 0s 136us/sample - loss: 0.4564 - acc: 0.7866
Epoch 9/10
3145/3145 [==============================] - 0s 133us/sample - loss: 0.4533 - acc: 0.7841
Epoch 10/10
3145/3145 [==============================] - 0s 73us/sample - loss: 0.4504 - ac

In [31]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.785078425890513 using {'batch_size': 10, 'epochs': 10}
Means: 0.785078425890513, Stdev: 0.009972977290012005 with: {'batch_size': 10, 'epochs': 10}
Means: 0.7651547317335089, Stdev: 0.02029360197978861 with: {'batch_size': 20, 'epochs': 10}
Means: 0.736964819718337, Stdev: 0.010642825629603767 with: {'batch_size': 40, 'epochs': 10}
Means: 0.736964819718337, Stdev: 0.010642825629603767 with: {'batch_size': 60, 'epochs': 10}
Means: 0.736964819718337, Stdev: 0.010642825629603767 with: {'batch_size': 80, 'epochs': 10}
Means: 0.736964819718337, Stdev: 0.010642825629603767 with: {'batch_size': 100, 'epochs': 10}


In [36]:
# Batch Size: 20

## Tune Optimizer

In [37]:
# Define optimizers
optimizers = ['adam', 'adagrad', 'adamax', 'nadam']

In [ ]:
# Loop over optimizers
for optimizer in optimizers:
    model = Sequential()
    model.add(Dense(4, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Complile Model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Fit model
    model.fit(X_train, y_train, batch_size=20, epochs=20)
    
    print('*********************')
    print(optimizer)
    model.evaluate(X_test, y_test)
    print('*********************')

In [133]:
# adam had the best score
# Model Building Function
def create_model():
    model = Sequential()
    model.add(Dense(4, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Complile Model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Tune Number of Neurons

In [34]:
neurons = [1, 2, 4, 8, 16, 32, 64, 128]

In [35]:
for neuron in neurons:
    model = Sequential()
    model.add(Dense(neuron, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Complile Model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fit
    model.fit(X_train, y_train, epochs=10)
    
    # Eval
    print('*****************')
    print(neuron)
    model.evaluate(X_test, y_test)
    print('*****************')

Epoch 1/10
4718/4718 [==============================] - 0s 98us/sample - loss: 0.6597 - acc: 0.7370
Epoch 2/10
4718/4718 [==============================] - 0s 63us/sample - loss: 0.6085 - acc: 0.7370
Epoch 3/10
4718/4718 [==============================] - 0s 58us/sample - loss: 0.5894 - acc: 0.7370
Epoch 4/10
4718/4718 [==============================] - 0s 56us/sample - loss: 0.5783 - acc: 0.7370
Epoch 5/10
4718/4718 [==============================] - 0s 55us/sample - loss: 0.5684 - acc: 0.7370
Epoch 6/10
4718/4718 [==============================] - 0s 51us/sample - loss: 0.5586 - acc: 0.7370
Epoch 7/10
4718/4718 [==============================] - 0s 59us/sample - loss: 0.5486 - acc: 0.7370
Epoch 8/10
4718/4718 [==============================] - 0s 56us/sample - loss: 0.5383 - acc: 0.7370
Epoch 9/10
4718/4718 [==============================] - 0s 57us/sample - loss: 0.5276 - acc: 0.7370
Epoch 10/10
4718/4718 [==============================] - 0s 59us/sample - loss: 0.5169 - acc: 0.7370

## Tune activation

In [42]:
# Define activation functions
activation_functions = ['relu', 'sigmoid', 'tanh', 'hard_sigmoid']

In [ ]:
# Loop over activation functions
for function in activation_functions:
    model = Sequential()
    model.add(Dense(32, input_dim=17, activation=function))
    model.add(Dense(1, activation='sigmoid'))
    
    # Complile Model
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    
    # Fit model
    model.fit(X_train, y_train, batch_size=20, epochs=20)
    
    print('*********************')
    print(function)
    model.evaluate(X_test, y_test)
    print('*********************')

In [ ]:
# relu

## Tune Layers

In [71]:
model = Sequential()
model.add(Dense(32, input_dim=17, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Complile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
model.fit(X_train, y_train, batch_size=20, epochs=20)

ValueError: Error when checking input: expected dense_125_input to have shape (17,) but got array with shape (18,)

In [48]:
model.evaluate(X_test, y_test)

2325/2325 [==============================] - 0s 176us/sample - loss: 0.4246 - acc: 0.8034


[0.42459832838786543, 0.80344087]

In [49]:
model = Sequential()
model.add(Dense(32, input_dim=17, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Complile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
model.fit(X_train, y_train, batch_size=20, epochs=20)

Epoch 1/20
4718/4718 [==============================] - 2s 374us/sample - loss: 0.5289 - acc: 0.7558
Epoch 2/20
4718/4718 [==============================] - 1s 162us/sample - loss: 0.4607 - acc: 0.7872
Epoch 3/20
4718/4718 [==============================] - 1s 158us/sample - loss: 0.4459 - acc: 0.7885
Epoch 4/20
4718/4718 [==============================] - 1s 160us/sample - loss: 0.4402 - acc: 0.7872
Epoch 5/20
4718/4718 [==============================] - 1s 167us/sample - loss: 0.4355 - acc: 0.7872
Epoch 6/20
4718/4718 [==============================] - 1s 159us/sample - loss: 0.4333 - acc: 0.7916
Epoch 7/20
4718/4718 [==============================] - 1s 169us/sample - loss: 0.4361 - acc: 0.7923
Epoch 8/20
4718/4718 [==============================] - 1s 169us/sample - loss: 0.4314 - acc: 0.7912
Epoch 9/20
4718/4718 [==============================] - 1s 290us/sample - loss: 0.4291 - acc: 0.7948
Epoch 10/20
4718/4718 [==============================] - 1s 169us/sample - loss: 0.4282 - a

In [51]:
model.evaluate(X_test, y_test)

2325/2325 [==============================] - 0s 183us/sample - loss: 0.4398 - acc: 0.7931


[0.4398345124336981, 0.7931183]

## Tune 

## Tune learning_rate

In [146]:
model = Sequential()
model.add(Dense(12, input_dim=17, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Complile Model
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X_train, y_train)

## Tune Epochs

In [151]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 50, 80, 100]}

In [ ]:
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [155]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7914370503426609 using {'batch_size': 20, 'epochs': 20}
Means: 0.7914370503426609, Stdev: 0.011002058099333254 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7895294619521229, Stdev: 0.006967281003188864 with: {'batch_size': 20, 'epochs': 50}
Means: 0.7899533762624578, Stdev: 0.008898200244804436 with: {'batch_size': 20, 'epochs': 80}
Means: 0.7865620941845933, Stdev: 0.006714299446677344 with: {'batch_size': 20, 'epochs': 100}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?